# Function Calling on FHIR with Knowledge Graphs

### 2. Neo4J & Jupyter Environment
This notebook needs an instance of [Neo4j](https://www.neo4j.com) to talk to. I used docker to run Neo4J locally using the following command:
```
docker run --name testneo4j -p7474:7474 -p7687:7687 -d \
    -v $HOME/neo4j/data:/data \
    -v $HOME/neo4j/logs:/logs \
    -v $HOME/neo4j/import:/var/lib/neo4j/import \
    -v $HOME/neo4j/plugins:/plugins \
    --env NEO4J_AUTH=neo4j/password \
    neo4j:latest
```
**Note:** No particular plugins are needed. 

You can also use a Neo4J Aurora instance. 

#### Jupyter Environment
Regardless of how you run Neo4J. You need to set some environment variables in the notebook's environment:

| Variable | Description | Value for above Docker |
|----------|-------------|------------------------|
| NEO4J_URL | Where to find the instance of Neo4j. | bolt://localhost:7687 |
| NEO4J_USER | The username for the database. | neo4j |
| NEO4J_PASSWORD | The password for the database. | password |


### 3. Synthetic data and working directory
The data I used for this notebook came from [Synthea](https://synthea.mitre.org/). Using the 

All the questions here us the FHIR Bundle: `fhir_data/stanfor_llm_on_fhir`

In [1]:
# Imports needed

import glob
import json
import os
import re

## Establish Database Connection

The cell connects to the Neo4J instance. It relies on several environment variables. 

**PLEASE NOTE**: The variable have been changed to support multiple databases in the same instance. 

| Variable            | Description                          | Sample Value          |
|---------------------|--------------------------------------|-----------------------|
| FHIR_GRAPH_URL      | Where to find the instance of Neo4j. | bolt://localhost:7687 |
| FHIR_GRAPH_USER     | The username for the database.       | neo4j                 |
| FHIR_GRAPH_PASSWORD | The password for the database.       | password              |
| FHIR_GRAPH_DATABASE | The name of the database instance.   | neo4j                 |

In [2]:
NEO4J_URI = os.getenv('FHIR_GRAPH_URL', 'neo4j://localhost:7687')
USERNAME = os.getenv('FHIR_GRAPH_USER', 'neo4j')
PASSWORD = os.getenv('FHIR_GRAPH_PASSWORD', 'password')
DATABASE = os.getenv('FHIR_GRAPH_DATABASE', 'neo4j')

## Helper Database Cells

The following three cells are here to be used to manage the database. They do not need to be run on a blank database. 

http://localhost:7474/browser/

### Create Function for Lexical Search 

This cell creates a new class and object for doing a lexical search on neo4j. 

This is here because running the cell above can take time and only should be done one time when the DB is created. 

In [3]:
from neo4j import GraphDatabase

class Neo4jResourceSearcher:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def search_by_resource_type(self, resource_type):
        with self.driver.session() as session:
            result = session.run("""
                MATCH (n)
                WHERE toLower(n.resource_type) CONTAINS toLower($resource_type)
                RETURN n.text AS text
            """, resource_type=resource_type)
            return [record["text"] for record in result]
    def search_by_resource_type_bfs(self, resource_type):
        with self.driver.session() as session:
            result = session.run("""MATCH (n)
                WHERE toLower(n.resource_type) CONTAINS toLower($resource_type)
                WITH n
                CALL {
                  // Perform breadth-first search (BFS) up to level 1 neighbors, excluding specific relationships
                  WITH n
                  MATCH (n)<-[r]->(neighbor)
                  WHERE NOT type(r) IN ['subject', 'patient'] // Exclude relationships of type 'subject' and 'patient'
                  RETURN neighbor, r
                }
                WITH n, collect(neighbor.text) AS neighbors
                RETURN n.text + " " + reduce(s = "", neighbor IN neighbors | s + "\n " + neighbor) AS text
                """, resource_type=resource_type)
            return [record["text"] for record in result]

    def search_by_resource_type_bfs_format(self, resource_type):
        with self.driver.session() as session:
            result = session.run("""MATCH (n)
WHERE toLower(n.resource_type) CONTAINS toLower($resource_type)
WITH n
CALL {
  // Perform breadth-first search (BFS) up to level 1 neighbors, excluding specific relationships
  WITH n
  MATCH (n)<-[r]->(neighbor)
  WHERE NOT type(r) IN ['subject', 'patient'] // Exclude relationships of type 'subject' and 'patient'
  RETURN neighbor, r
}
WITH n, collect({neighbor_text: neighbor.text, relationship: type(r)}) AS neighbors
RETURN n.text + "\nNeighbors:\n" + 
       reduce(s = "", neighbor IN neighbors | s + "Relationship: " + neighbor.relationship + "\nResource: " + neighbor.neighbor_text + "\n") AS text
            """, resource_type=resource_type)
            return [record["text"] for record in result]


In [4]:
def get_resources(resource_type: str) -> str:
    """Take a FHIR R4 Resource type and searh in the database the resources available.
    Every Medication request must be redirect to MedicationRequest
    
    Args:
        resource_type: Valid possible values: i.e. Patient, Practitioner, PractitionerRole, Organization, Encounter, Observation, Condition, Procedure, Medication, MedicationRequest, Immunization, DiagnosticReport, AllergyIntolerance, CarePlan, CareTeam, Appointment, Coverage, Claim, Device, DocumentReference
    Output:
        string of the resources for a giving resource_type
    """
    searcher = Neo4jResourceSearcher(NEO4J_URI, DATABASE, PASSWORD)

    results = searcher.search_by_resource_type_bfs_format(resource_type)
    return len(results)

print(get_resources("MedicationRequest"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 4, column: 1, offset: 81} for query: 'MATCH (n)\nWHERE toLower(n.resource_type) CONTAINS toLower($resource_type)\nWITH n\nCALL {\n  // Perform breadth-first search (BFS) up to level 1 neighbors, excluding specific relationships\n  WITH n\n  MATCH (n)<-[r]->(neighbor)\n  WHERE NOT type(r) IN [\'subject\', \'patient\'] // Exclude relationships of type \'subject\' and \'patient\'\n  RETURN neighbor, r\n}\nWITH n, collect({neighbor_text: neighbor.text, relationship: type(r)}) AS neighbors\nRETURN n.text + "\nNeighbors:\n" + \n       reduce(s = "", neighbor IN neighbors | s + "Relationship: " + neighbor.relationship + "\nResource: " + neighbor.neighbor

6


In [5]:
def get_resources(resource_type: str) -> str:
    """Take a FHIR R4 Resource type and searh in the database the resources available.
    Every Medication request must be redirect to MedicationRequest
    
    Args:
        resource_type: Valid possible values: i.e. Patient, Practitioner, PractitionerRole, Organization, Encounter, Observation, Condition, Procedure, Medication, MedicationRequest, Immunization, DiagnosticReport, AllergyIntolerance, CarePlan, CareTeam, Appointment, Coverage, Claim, Device, DocumentReference
    Output:
        string of the resources for a giving resource_type
    """
    searcher = Neo4jResourceSearcher(NEO4J_URI, DATABASE, PASSWORD)

    results = searcher.search_by_resource_type_bfs(resource_type)
    return len(results)

print(get_resources("MedicationRequest"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 4, column: 17, offset: 129} for query: 'MATCH (n)\n                WHERE toLower(n.resource_type) CONTAINS toLower($resource_type)\n                WITH n\n                CALL {\n                  // Perform breadth-first search (BFS) up to level 1 neighbors, excluding specific relationships\n                  WITH n\n                  MATCH (n)<-[r]->(neighbor)\n                  WHERE NOT type(r) IN [\'subject\', \'patient\'] // Exclude relationships of type \'subject\' and \'patient\'\n                  RETURN neighbor, r\n                }\n                WITH n, collect(neighbor.text) AS neighbors\n                RETURN n.text + " " + redu

6


In [26]:
from langchain_core.tools import tool

@tool
def get_resources(resource_type: str) -> str:
    """Take a FHIR R4 Resource type and searh in the database the resources available.
    
    Every Medication related request must be redirect to MedicationRequest
    Every Doctor related request must be redirect to Practitioner
    
    Args:
        resource_type: Valid possible values: i.e. Patient, Practitioner, PractitionerRole, Organization, Encounter, Observation, Condition, Procedure, Medication, MedicationRequest, Immunization, DiagnosticReport, AllergyIntolerance, CarePlan, CareTeam, Appointment, Coverage, Claim, Device, DocumentReference
    Output:
        string of the resources for a giving resource_type
    """
    searcher = Neo4jResourceSearcher(NEO4J_URI, DATABASE, PASSWORD)

    results = searcher.search_by_resource_type(resource_type)
    return "\n".join(results)

@tool
def get_resources_expanded(resource_type: str) -> str:
    """Take a FHIR R4 Resource type and searh in the database the resources available.

    Every Medication related request must be redirect to MedicationRequest
    Every Doctor related request must be redirect to Practitioner
    
    Args:
        resource_type: Valid possible values: i.e. Patient, Practitioner, PractitionerRole, Organization, Encounter, Observation, Condition, Procedure, Medication, MedicationRequest, Immunization, DiagnosticReport, AllergyIntolerance, CarePlan, CareTeam, Appointment, Coverage, Claim, Device, DocumentReference
    Output:
        string of the resources for a giving resource_type
    """
    searcher = Neo4jResourceSearcher(NEO4J_URI, DATABASE, PASSWORD)

    results = searcher.search_by_resource_type_bfs_format(resource_type)
    return "\n".join(results)

In [27]:
tools = [get_resources]

### Explorar later

from langchain_core.pydantic_v1 import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]

## Gemini PRO 1.5

In [28]:
from IPython.display import display, Markdown

Bind the function in the LLM

In [29]:
from langchain_google_vertexai import VertexAI
from langchain_google_vertexai import ChatVertexAI
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage

llm = ChatVertexAI(model="gemini-1.5-pro-002", temperature=0)

#### Non Detail answer

In [30]:
tools = [get_resources]

In [31]:
llm_with_tools = llm.bind_tools(tools)

In [33]:
query = "What doctors I ever vistir in my life ?"

In [34]:
messages = [
    SystemMessage("""You are a highly skilled FHIR Specialist tasked with thoroughly analyzing patient queries. Your goal is to provide comprehensive, detailed responses based on the full context provided.

When answering, ensure your responses are rich in specific details such as locations, costs, timelines, and any other relevant information that would be valuable to the patient. Consider the broader implications and include any additional insights that may assist the patient in making informed decisions.
Before responding, always analyze the entire context to ensure your answer is both accurate and exhaustive. Aim for clarity and depth in your explanations."""),
    HumanMessage(query)
]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_resources": get_resources}[tool_call["name"].lower()]
    tool_output = selected_tool(tool_call["args"]["resource_type"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [35]:
messages

[SystemMessage(content='You are a highly skilled FHIR Specialist tasked with thoroughly analyzing patient queries. Your goal is to provide comprehensive, detailed responses based on the full context provided.\n\nWhen answering, ensure your responses are rich in specific details such as locations, costs, timelines, and any other relevant information that would be valuable to the patient. Consider the broader implications and include any additional insights that may assist the patient in making informed decisions.\nBefore responding, always analyze the entire context to ensure your answer is both accurate and exhaustive. Aim for clarity and depth in your explanations.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What doctors I ever vistir in my life ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_resources', 'arguments': '{"resource_type": "Encounter"}'}}, response_metadata={'is_blocked': F

In [36]:
response = llm_with_tools.invoke(messages)

In [37]:
display(Markdown(response.content))

You have visited two doctors: Dr. Elisa944 Rojo930 and Dr. Alvin56 Crona259.

Dr. Elisa944 Rojo930 practices at BLUESKIES WELLNESS INC located at https://github.com/synthetichealth/synthea|97876511-009c-391a-a262-1557e46861bb. You have seen this doctor multiple times between 2015 and 2023, mostly for well-child visits.  

Dr. Alvin56 Crona259 practices at HOLY FAMILY HOSPITAL located at https://github.com/synthetichealth/synthea|17ab5f4a-eb74-306c-8895-5ce93ec4592b. Your visits with this doctor occurred between 2015 and 2023 for various reasons, including atopic dermatitis, acute viral pharyngitis, otitis media, perennial allergic rhinitis, an allergy to a substance, strep throat, and check-ups.  You also saw Dr. Ted955 Reilly981 at Fitchburg Outpatient Clinic for Aortic valve stenosis.


In [54]:
query = "What are my current medications and how should I be taking them? detail_anwser=True"

In [56]:
messages = [
    SystemMessage("""You are a highly skilled FHIR Specialist tasked with thoroughly analyzing patient queries. Your goal is to provide comprehensive, detailed responses based on the full context provided.

When answering, ensure your responses are rich in specific details such as locations, costs, timelines, and any other relevant information that would be valuable to the patient. Consider the broader implications and include any additional insights that may assist the patient in making informed decisions.
Before responding, always analyze the entire context to ensure your answer is both accurate and exhaustive. Aim for clarity and depth in your explanations."""),
    HumanMessage(query)
]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_resources": get_resources}[tool_call["name"].lower()]
    tool_output = selected_tool(tool_call["args"]["resource_type"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [58]:
response = llm_with_tools.invoke(messages)

In [59]:
display(Markdown(response.content))

Here's a summary of your current and past medications, along with details on how they should be taken:

**Active Medications:**

* **Fexofenadine hydrochloride 30 MG Oral Tablet:** Take as needed. This medication was prescribed on 2017-08-22 by Dr. Alvin56 Crona259.
* **NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Injector:**  Take as needed. This was prescribed on 2017-08-30 by Dr. Alvin56 Crona259.

**Past Medications:**

* **Amoxicillin 500 MG Oral Tablet:**  Take at regular intervals. Complete the prescribed course unless otherwise directed. The dosage is 1 tablet, 3 times per day. This medication was prescribed on 2016-11-23 by Dr. Alvin56 Crona259 and is now stopped.
* **Acetaminophen 160 MG Chewable Tablet:** Take as needed. This was prescribed on 2016-11-23 by Dr. Alvin56 Crona259 and is now stopped.
* **Penicillin V Potassium 250 MG Oral Tablet (2 Prescriptions):**  This medication was prescribed twice by Dr. Alvin56 Crona259 for Streptococcal sore throat, once on 2021-02-17 and again on 2022-02-26. Both prescriptions are now stopped.  Dosage instructions are not specified in the records.

Please note:  Always consult with your doctor or pharmacist if you have any questions or concerns about your medications. This information is based on your medical records and may not be completely up-to-date.  It's important to verify current medications and dosages with your healthcare provider.


### Detail answer

In [99]:
tools = [get_resources_expanded]

In [100]:
llm_with_tools = llm.bind_tools(tools)

In [102]:
query = "What are my current medications and how should I be taking them ?"

messages = [
    SystemMessage("""You are a highly skilled FHIR Specialist tasked with thoroughly analyzing patient queries. Your goal is to provide comprehensive, detailed responses based on the full context provided.
When answering, ensure your responses are rich in specific details such as dates, doctors, providers, locations, billing and any other relevant information that would be valuable to the patient. Consider the broader implications and include any additional insights that may assist the patient in making informed decisions.
Before responding, always analyze the entire context to ensure your answer is both accurate and exhaustive. Aim for clarity and depth in your explanations."""),
    HumanMessage(query)
]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_resources_expanded": get_resources_expanded}[tool_call["name"].lower()]
    tool_output = selected_tool(tool_call["args"]["resource_type"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

response = llm_with_tools.invoke(messages)
display(Markdown(response.content))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 4, column: 1, offset: 81} for query: 'MATCH (n)\nWHERE toLower(n.resource_type) CONTAINS toLower($resource_type)\nWITH n\nCALL {\n  // Perform breadth-first search (BFS) up to level 1 neighbors, excluding specific relationships\n  WITH n\n  MATCH (n)<-[r]->(neighbor)\n  WHERE NOT type(r) IN [\'subject\', \'patient\'] // Exclude relationships of type \'subject\' and \'patient\'\n  RETURN neighbor, r\n}\nWITH n, collect({neighbor_text: neighbor.text, relationship: type(r)}) AS neighbors\nRETURN n.text + "\nNeighbors:\n" + \n       reduce(s = "", neighbor IN neighbors | s + "Relationship: " + neighbor.relationship + "\nResource: " + neighbor.neighbor

Here's a summary of your current and past medications, including how they were prescribed:

* **Active Medications:**
    * **Fexofenadine hydrochloride 30 MG Oral Tablet:** Take as needed for perennial allergic rhinitis. This was prescribed on August 22nd, 2017, by Dr. Alvin56 Crona259 during a visit to HOLY FAMILY HOSPITAL.  The associated claim for this prescription totalled $455.32.
    * **NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Injector:**  Take as needed for allergy to substance. Prescribed on August 30th, 2017, by Dr. Alvin56 Crona259 during a visit to HOLY FAMILY HOSPITAL. The claim for this auto-injector was $64.66.

* **Past Medications (Stopped):**
    * **Amoxicillin 500 MG Oral Tablet:**  Taken 3 times a day for a completed course, prescribed for Otitis media (ear infection) on November 23rd, 2016, by Dr. Alvin56 Crona259 during a visit to HOLY FAMILY HOSPITAL. The claim was $103.33.
    * **Acetaminophen 160 MG Chewable Tablet:** Taken as needed, prescribed on November 23, 2016 by Dr. Alvin56 Crona259 during a visit for Otitis media at HOLY FAMILY HOSPITAL. The associated claim was $91.78.
    * **Penicillin V Potassium 250 MG Oral Tablet:** Prescribed twice, both times by Dr. Alvin56 Crona259 at HOLY FAMILY HOSPITAL for Streptococcal sore throat:
        * First prescription on February 17th, 2021 with a claim of $478.69.
        * Second prescription on February 26th, 2022 with a claim of $324.32.


It's important to note that this information is based on past records.  Always consult with your doctor or pharmacist for the most up-to-date information about your medications and how to take them.  Do not adjust your medication or stop taking it without professional guidance.


In [104]:
response.content

"Here's a summary of your current and past medications, including how they were prescribed:\n\n* **Active Medications:**\n    * **Fexofenadine hydrochloride 30 MG Oral Tablet:** Take as needed for perennial allergic rhinitis. This was prescribed on August 22nd, 2017, by Dr. Alvin56 Crona259 during a visit to HOLY FAMILY HOSPITAL.  The associated claim for this prescription totalled $455.32.\n    * **NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Injector:**  Take as needed for allergy to substance. Prescribed on August 30th, 2017, by Dr. Alvin56 Crona259 during a visit to HOLY FAMILY HOSPITAL. The claim for this auto-injector was $64.66.\n\n* **Past Medications (Stopped):**\n    * **Amoxicillin 500 MG Oral Tablet:**  Taken 3 times a day for a completed course, prescribed for Otitis media (ear infection) on November 23rd, 2016, by Dr. Alvin56 Crona259 during a visit to HOLY FAMILY HOSPITAL. The claim was $103.33.\n    * **Acetaminophen 160 MG Chewable Tablet:** Taken as needed, prescribed on

In [106]:
import pprint

print(response.content)

Here's a summary of your current and past medications, including how they were prescribed:

* **Active Medications:**
    * **Fexofenadine hydrochloride 30 MG Oral Tablet:** Take as needed for perennial allergic rhinitis. This was prescribed on August 22nd, 2017, by Dr. Alvin56 Crona259 during a visit to HOLY FAMILY HOSPITAL.  The associated claim for this prescription totalled $455.32.
    * **NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Injector:**  Take as needed for allergy to substance. Prescribed on August 30th, 2017, by Dr. Alvin56 Crona259 during a visit to HOLY FAMILY HOSPITAL. The claim for this auto-injector was $64.66.

* **Past Medications (Stopped):**
    * **Amoxicillin 500 MG Oral Tablet:**  Taken 3 times a day for a completed course, prescribed for Otitis media (ear infection) on November 23rd, 2016, by Dr. Alvin56 Crona259 during a visit to HOLY FAMILY HOSPITAL. The claim was $103.33.
    * **Acetaminophen 160 MG Chewable Tablet:** Taken as needed, prescribed on November

### Highlight costs

In [101]:
query = "What are my current medications and how did cost them ?"

messages = [
    SystemMessage("""You are a highly skilled FHIR Specialist tasked with thoroughly analyzing patient queries. Your goal is to provide comprehensive, detailed responses based on the full context provided.
When answering, ensure your responses are rich in specific details such as dates, doctors, providers, locations, billing and any other relevant information that would be valuable to the patient. Consider the broader implications and include any additional insights that may assist the patient in making informed decisions.
Before responding, always analyze the entire context to ensure your answer is both accurate and exhaustive. Aim for clarity and depth in your explanations."""),
    HumanMessage(query)
]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_resources_expanded": get_resources_expanded}[tool_call["name"].lower()]
    tool_output = selected_tool(tool_call["args"]["resource_type"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

response = llm_with_tools.invoke(messages)
display(Markdown(response.content))

This model can reply with multiple function calls in one response. Please don't rely on `additional_kwargs.function_call` as only the last one will be saved.Use `tool_calls` instead.
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 4, column: 1, offset: 81} for query: 'MATCH (n)\nWHERE toLower(n.resource_type) CONTAINS toLower($resource_type)\nWITH n\nCALL {\n  // Perform breadth-first search (BFS) up to level 1 neighbors, excluding specific relationships\n  WITH n\n  MATCH (n)<-[r]->(neighbor)\n  WHERE NOT type(r) IN [\'subject\', \'patient\'] // Exclude relationships of type \'subject\' and \'patient\'\n  RETURN neighbor, r\n}\nWITH n, collect({neighbor_text: neighbor.text, relationship: type(

Your current medications include Fexofenadine hydrochloride 30 MG Oral Tablet and NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Injector. Both were prescribed by Dr. Alvin56 Crona259 and are to be taken as needed.  The Fexofenadine was prescribed on 2017-08-22 and cost $455.32, while the Epinephrine was prescribed on 2017-08-30 and cost $64.66.  These prescriptions were filled at HOLY FAMILY HOSPITAL and billed to your Medicare coverage.  You also had prescriptions for Amoxicillin 500 MG Oral Tablet and Acetaminophen 160 MG Chewable Tablet, both prescribed on 2016-11-23 by the same doctor for Otitis media, costing $103.33 and $91.78 respectively. Finally, a Penicillin V Potassium 250 MG Oral Tablet prescription was filled twice, costing $478.69 on 2021-02-17 and $324.32 on 2022-02-26, both related to Streptococcal sore throat.


## Conditions Summarization 

In [82]:
messages = [
    SystemMessage("""You are a highly skilled FHIR Specialist tasked with thoroughly analyzing patient queries. Your goal is to provide comprehensive, detailed responses based on the full context provided.

When answering, ensure your responses are rich in specific details such as locations, costs, timelines, and any other relevant information that would be valuable to the patient. Consider the broader implications and include any additional insights that may assist the patient in making informed decisions.
Before responding, always analyze the entire context to ensure your answer is both accurate and exhaustive. Aim for clarity and depth in your explanations."""),
    HumanMessage("Summarize my conditions in details")
]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_resources": get_resources}[tool_call["name"].lower()]
    tool_output = selected_tool(tool_call["args"]["resource_type"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [83]:
response = llm_with_tools.invoke(messages)

In [84]:
display(Markdown(response.content))

This is a summary of your conditions:

* **Medication review due (situation)**: This condition has been recorded multiple times throughout your history, indicating recurring needs for medication reviews.  The instances were:
    * Resolved on May 3, 2015, associated with encounter 'b3b4d7c6-46f5-9f10-cfd2-464ff96160f9'.
    * Resolved on June 7, 2015, associated with encounter 'd7a2721e-1568-274b-64bb-58c2ec026b65'.
    * Resolved on January 10, 2016 (onset August 9, 2015), associated with encounter 'fd013f8c-0ce2-644e-1d71-bf6761396ba8'.
    * Resolved on July 10, 2016 (onset April 10, 2016), associated with encounter '35c0f7d7-4502-8834-8202-a8f771c3e525'.
    * Resolved on March 6, 2022 (onset October 9, 2016), associated with encounter 'f48e195a-c086-b88c-ba69-eb7bd59c2111'.
    * Resolved on April 24, 2022, associated with encounter 'def99498-60f2-be13-28c1-89067e3cda07'.
    * Resolved on April 30, 2023, associated with encounter '48f3913f-2909-1547-f860-788b455b7951'.

* **Atopic dermatitis**: This is an active condition recorded on September 13, 2015, associated with encounter 'da622f16-5f38-de7a-6801-e0bf78546be5'.  Atopic dermatitis is a chronic skin condition causing inflammation and itchiness.  Its active status suggests ongoing management.

* **Acute viral pharyngitis (disorder)**:  There are two resolved instances of this condition, commonly known as a sore throat:
    * Resolved on October 9, 2016 (onset September 26, 2016), associated with encounter 'a9e6308f-927d-a7fa-c801-680fc1e3e6ff'.
    * Resolved on October 14, 2023 (onset October 2, 2023), associated with encounter '8587fc98-74aa-c333-61b1-d2ad195941eb'.

* **Otitis media**: This condition, commonly known as an ear infection, was resolved on April 9, 2017 (onset November 23, 2016), associated with encounter 'cf60427f-6701-abf2-82fc-945837df2152'.

* **Perennial allergic rhinitis**: This is an active condition recorded on August 22, 2017, associated with encounter '9fc6351d-7e69-20c6-c597-856f8e70483d'.  Perennial allergic rhinitis is a type of chronic allergy affecting the nose.  Its active status indicates ongoing symptoms and the need for management.

* **Aortic valve stenosis (disorder)**: This is an active condition recorded on May 2, 2019, associated with encounter '5e225a66-ead5-6de3-bba6-bb6be422159d'. This is a serious heart condition involving the narrowing of the aortic valve, restricting blood flow.  Its active status indicates ongoing monitoring and potential treatment.

* **Streptococcal sore throat (disorder)**:  A resolved case of strep throat was recorded on February 26, 2021 (onset February 17, 2021), associated with encounter '4e7837d4-083f-9360-e1e3-3550ea57d9d9'. Another resolved case was recorded on March 9, 2022 (onset February 26, 2022), associated with encounter 'fc6a22f2-848e-d29e-0a8b-df2e931ae617'.  Strep throat is a bacterial infection.


It's crucial to discuss the active conditions—Atopic dermatitis, Perennial allergic rhinitis, and Aortic valve stenosis—with your healthcare provider to ensure appropriate ongoing management.  While the other conditions are resolved, they provide valuable context to your overall health history.  Make sure to mention any recurring symptoms or concerns to your physician.


## Conditions Summarization - Active

In [85]:
messages = [
    SystemMessage("""You are a highly skilled FHIR Specialist tasked with thoroughly analyzing patient queries. Your goal is to provide comprehensive, detailed responses based on the full context provided.

When answering, ensure your responses are rich in specific details such as locations, costs, timelines, and any other relevant information that would be valuable to the patient. Consider the broader implications and include any additional insights that may assist the patient in making informed decisions.
Before responding, always analyze the entire context to ensure your answer is both accurate and exhaustive. Aim for clarity and depth in your explanations."""),
    HumanMessage("Summarize my active conditions in details")
]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_resources": get_resources}[tool_call["name"].lower()]
    tool_output = selected_tool(tool_call["args"]["resource_type"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [86]:
response = llm_with_tools.invoke(messages)

In [87]:
display(Markdown(response.content))

You currently have three active conditions:

1. **Atopic dermatitis:** This was diagnosed on September 13, 2015.  It is a chronic skin condition causing dry, itchy, and inflamed skin.  There's no information on current treatments or medications prescribed for this condition in the provided data.  It would be beneficial to discuss management options with your healthcare provider.

2. **Perennial allergic rhinitis:** Diagnosed on August 22, 2017. This is a persistent form of allergic rhinitis,  meaning your symptoms occur year-round rather than seasonally.  Common triggers include dust mites, pet dander, and mold.  The provided data does not include details about specific allergens or treatments.  Discussing allergy testing and management strategies with your doctor could improve your quality of life.

3. **Aortic valve stenosis:** This was diagnosed on May 2, 2019.  This condition involves the narrowing of the aortic valve, which restricts blood flow from the heart to the rest of the body. Depending on the severity, treatments can range from medication to surgical intervention. The data provided doesn't specify the severity or any ongoing treatments for this condition. It's important to have regular check-ups with your cardiologist to monitor the progression of this condition and discuss appropriate treatment strategies.


It's important to note that several other conditions are listed as resolved. These include instances of "Medication review due," acute viral pharyngitis, otitis media, and streptococcal sore throat.  While these conditions are no longer active, their history is relevant to your overall health record.
